In [1]:
import os
import openai
import json
import random
import time
import copy
import torch
import utils
from transformers import AutoTokenizer,AutoConfig, AutoModel
openai.api_base=''
openai.api_key = ''    #这个用closeai的接口，不需要翻墙

/home/jiaoyilin/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open('../data/carb_50.txt','r') as f:     
    temp_sent=f.readlines()
#with open('../data/RE-OIE2016_50.txt','r') as f:     
#    temp_sent=f.readlines()
#with open('data/RE-OIE2016_50_100.txt','r') as f:     
#    temp_sent1=f.readlines()
#with open('data/RE-OIE2016_100_200.txt','r') as f:   
#    temp_sent2=f.readlines()

In [4]:
task1='Extract the triples which consist of subject, predicate and object from the sentence. '
task2='Extract subject-object pairs and the predicate between them in the sentence. Subject-object pairs and predicate should form triples. '
task3='Extract the triples which consist of subject, object and the relation between them from the sentence. '
task4='Extract two types of triples in the sentence. The first type of triples consist of subject, predicate and object. \
The second type of triples consist of subject, predicate, predicative. '
task5='Extract the coherent propositions from a sentence, each comprising of a predicate phrase and two argument phrases. '
format1='Output the triples as <subject|predicate|object> and use ; to split the triples. '
format2='Print each result as the format Subject-object pairs:(subject,object);Predicate: predicate\n.'
format3='Output the triples as <subject|relation|object> and use ; to split the triples. '
format4='Output the triples as <subject|predicate|object> and <subject|predicate|predicative> and use ; to split the triples. '
format5='Output the propositions as <argument1|predicate|argument2> and use ; to split the propostions. \
The predicate of some propositions may not be in the sentence, add them. Here is the sentences:'

In [5]:
message1=[{"role": "system", "content": task1+format1}] 
message2=[{"role": "system", "content": task2+format2}] 
message3=[{"role": "system", "content": task3+format3}] 
message4=[{"role": "system", "content": task4+format4}] 
message5=[{"role": "system", "content": task5+format5}] 

In [ ]:
result=[]
for data in temp_sent:
    message3.append({"role": "user", "content": data})
    temp_result=utils.gptresult(message3)
    result.append(temp_result)

x=utils.write_result(result,'../result/zero-shot/CaRB/task3.carb',temp_sent)

In [44]:

request1='Note that the triples which consist of subject, linking verb, and predicate adjective are also need to be extracted. '
request2='Extract as many triples as possible from the sentence and think carefully. ' 
request3='Note that adjectives can also be used as argument.'
request4='For example,in the sentence "she is beautiful.", "beautiful" can be a argument in it. The propositions should be <She|is|beautiful>. '

In [45]:
message6=[{"role": "system", "content": task1+request1+new_format}]  
message7=[{"role": "system", "content": task1+request1+new_format+request2}]
message8=[{"role": "system", "content": task1+request3+format5}]   
message9=[{"role": "system", "content": task1+request3+request4+format5}]

In [ ]:
result=[]
for data in temp_sent:
    message8.append({"role": "user", "content": data})
    temp_result=utils.gptresult(message8)
    result.append(temp_result)
x=utils.write_result(result,'../result/zero-shot/CaRB/request3.carb',temp_sent)


In [5]:
role1='You are a open information extractor. '       #两个role差别不大
role2='You are a open relation extractor. '
role3='You are an expert in the field of knowledge graph construction. '
role4='You are a linguist.'
role5='You are a propositional scientist.'
role6='I want you to act as a sentence information extractor.'
aim='You extract these propositions in order to build a knowledge graph. \
The points in the graph are the arguments in the proposition and the edges in the graph are the relation between these parameters. '
magic='Let us think step by step.'
magic2='Let us work this out in a step by step way to be sure we have the right answer.'
message4=[{"role": "system", "content": role1+task1+format1}] 
message5=[{"role": "system", "content": role2+task1+format1}] 
message6=[{"role": "system", "content": role1+task1+aim+format1}] 
message7=[{"role": "system", "content": role3+task1+aim+format1}] 
message8=[{"role": "system", "content": role4+task1+format1}] 
message9=[{"role": "system", "content": role5+task1+format1}] 
message10=[{"role": "system", "content": task1+format1+magic}]
message11=[{"role": "system", "content": task1+format1+magic2}]
message12=[{"role": "system", "content": role6+task1+format1}]

In [ ]:
result=[]
for data in temp_sent:
    message12.append({"role": "user", "content": data})
    temp_result=utils.gptresult(message12)
    result.append(temp_result)
x=utils.write_result(result,'../result/zero-shot/CaRB/role6.carb',temp_sent)

In [13]:
generate_prompt1='I want you to act as a ChatGPT prompt generator. I will send a topic,\
you have to generate a ChatGPT prompt based on the content of the topic. \
The prompt should start with “I want you to act as “, and guess what I might do, \
and expand the prompt accordingly Describe the content to make it useful.\
The topic is: Extract the triples which consist of subject, predicate and object from the sentence.\
And the result need to be the format: <subject|predicate|object>.'

generate_prompt2='I want you to act as a prompt generator. \
Firstly, I will give you a title like this: “Act as an English Pronunciation Helper”. \
Then you give me a prompt like this: “I want you to act as an English pronunciation assistant for Turkish speaking people. \
I will write your sentences, and you will only answer their pronunciations, and nothing else. \
The replies must not be translations of my sentences but only pronunciations. \
Pronunciations should use Turkish Latin letters for phonetics. Do not write explanations on replies. \
My first sentence is “how the weather is in Istanbul?”.” (You should adapt the sample prompt according to the title I gave. \
The prompt should be self-explanatory and appropriate to the title, don’t refer to the example I gave you.). '
prompt1='My first title is “Act as a sentence propositions extractor” (Give me prompt only)'
prompt2='My first title is “Act as a sentence <subject|predicate|object> triples extractor” (Give me prompt only)'

enhance_prompt='Act as a Prompt Enhancer AI that takes user-input prompts and transforms them into more engaging, detailed, and thought-provoking prompts. '


In [53]:
message=[{"role": "system", "content": enhance_prompt}] 
temp_prompt='Extract the triples which consist of subject, predicate and object from the sentence. \
There is no need to extract the adverbs of sentence in the extraction process.\
Output the triples as <subject |predicate |object> and use ‘;’ to split the triples.'
message.append({"role": "user", "content": temp_prompt})
concept_test = openai.ChatCompletion.create(model="gpt-4",messages=message,temperature=0)
new_prompt=concept_test['choices'][0]['message']['content']
new_prompt

'Develop a method to identify and extract the core components - subject, predicate, and object - from a given sentence. Your focus should be solely on these three elements, disregarding any adverbs present in the sentence. Present your findings in the format <subject | predicate | object>, using a semicolon to separate each distinct triple. This task will test your understanding of sentence structure and your ability to isolate key elements within a complex context.'

In [ ]:
%%time
generate_message=[{"role": "system", "content": new_prompt}]
result=[]
for data in temp_sent:
    generate_message.append({"role": "user", "content": data})
    temp_result=utils.gptresult(generate_message)
    result.append(temp_result)
utils.write_result(result,'../result/zero-shot/CaRB/enhance_task1.txt',temp_sent)